# 程序编程中的解决方法

 **简化问题**
* 将问题清楚重述, 思考如何将问题化简为多步解决, 搞清楚每一步需要做什么, 如何做做到解决每一小步的问题, 最后如何串联起每一步, 将整个问题整体解决.

**寻求帮助**
* 在网上寻求 $\text{Google}$ 的帮助, 在简化问题的基础上, 去查询自己需要解决的问题是什么, 目的明确, 在 $\text{Google}$ 中查找问题,需要掌握, 用英语怎么提问, 怎么去查找, $\text{Google}$ 中的资料更为完善, 除此之外, 还可以采用 **百度**,诸如这样的提问网站 `https://meta.stackoverflow.com/questions` 等. 

$\textbf{例如:}$用英语提问:

`How to rotate a square numpy array with different times efficiently by `np.rot90`?`

![scale=0.1](./figures/solve-problem.png)

### 对于网格生成的简单举例

####  以三角形网格为例

 **网格的数据结构**

* 网格节点坐标数组 `point`:
    + $N\times 2$ 的二维数组
    + `point[i, 0]` 和 `point[i, 1]` 分别存储第 $i$ 个网格节点的 $x$ 和 $y$ 坐标
* 单元顶点编号数组 `cell`:
    + $NC\times 3$ 的二维数组
    + `cell[i, 0]`, `cell[i, 1]` 和 `cell[i, 2]` 分别存储第 $i$ 个单元三个顶点的全局编号(即 `point` 中的行号)

In [2]:
import numpy as np
point = np.array(
    [(0.0, 0.0),
     (1.0, 0.0),
     (1.0, 1.0),
     (0.0, 1.0)], dtype=np.float)
cell = np.array([
        (1, 2, 0), 
        (3, 0, 2)], dtype=np.int)

#### 调用三角形网格生成算法 $\text{fealpy}$ 算法包

* 导入三角形网格生成算法.

In [ ]:
from fealpy.mesh.TriangleMesh import TriangleMesh

调用 $\text{TriangleMesh}$ 算法生成三角形网格

In [ ]:
tmesh = TriangleMesh(point, cell)

网格生成可以获得的信息, 可以采用加密网格生成算法.

In [ ]:
tmesh.point  #网格的顶点
tmesh.ds.edge #网格的边
tmesh.ds.cell #网格的单元
tmesh.ds.edge2cell
tmesh.ds.cell_to_edge()

如何将网格生成, 呈现出来.

![scale=0.1](./figures/solve-problem.png)导入画图模块

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
fig = plt.figure()# 生成画布
axes = fig.gca()# 生成坐标系
tmesh.add_plot(axes)#画图生成网格
tmesh.find_point(axes, showindex=True) #显示网格顶点编号
tmesh.find_edge(axes, showindex=True)# 显示网格边编号
tmesh.find_cell(axes, showindex=True)# 显示网格单元编号
plt.savefig("test.png")# 保存图
plt.show() # 显示网格

![scale=0.1](./figures/test.png)


### 对于一个具体的有限元程序的编程举例

* 有限元问题系统的理论及编程过程

 **问题**： 

\begin{equation}
-\Delta u =f\quad{(1)}
\end{equation}

有限元的基本问题可以归纳为:

* 把问题转化成变分形式
* 选定单元的形状, 对求解域做剖分. 一维情形的单元是小区间, 二维情形的重要单元有两种, 三角形和四边形(矩形和任意凸四边形). 三维情形的单元就更复杂, 四面体,六面体等.
* 构造基函数或单元形状函数, 行成有限元空间.
* 形成有限元方程, $Ax=b$.
* 提供有限元方程的有效解法.
* 收敛性及误差估计.

**解决问题:**

### 问题的变分

引入函数空间 $H^1(\Omega)$, 对于任意 $v \in H^1(\Omega)$, $v$ 和它的一阶导数都在 $\Omega$ 上 $L^2$ 可积. 这里的 $H^1(\Omega)$ 是一个无限维的空间.

另外, 引入空间 $H_0^1(\Omega) := \{v\in H^1(\Omega), v|_{\partial\Omega} = 0\}$.

对于任意的 $v\in H^1_0(\Omega)$, 同乘以方程 (1) 的两端, 然后做分部积分可得: 

\begin{equation}
\int_{\Omega}\nabla u\cdot\nabla v\mathrm{d}x = \int_{\Omega}fv\mathrm{d}x,\quad\forall v \in H^1_0(\Omega).\quad{(2)}
\end{equation}

引入双线性泛函

\begin{equation}
a(u,v) = <f,v> \text{ for all }v\in H_0^1(\Omega).\quad{(3)}
\end{equation}

$$
a(u,v) = \int_{\Omega}\nabla u\cdot\nabla v\mathrm{d}x,\quad <f,v> =  \int_{\Omega}fv\mathrm{d}x.
$$

1. 变分形式
$$
(\nabla u,\nabla v)=(f,v)
$$

2. 离散形式

$$
(\nabla u_h,\nabla v_h)=(f,v_h)
$$

### 网格的选取(三角形网格)

**二维情形**

* 选择均匀网格加密.

**准备初始网格**